In [6]:
# --- Install dependencies if needed ---
# !pip install pandas requests

import json
import pandas as pd
import requests

# ---------- Load Data from API ----------
url = "https://api.fablabs.io/0/labs.json"
response = requests.get(url)
labs_data = response.json()  # directly parse JSON

# ---------- Normalize Data ----------
df = pd.json_normalize(labs_data)

# ---------- Clean Data ----------
df.fillna("", inplace=True)

columns = [
    "id", "name", "city", "county", "country_code", 
    "activity_status", "capabilities", "latitude", "longitude"
]
df = df[columns]

df["capabilities"] = df["capabilities"].apply(lambda x: x if isinstance(x, list) else [])

df.head()


C:\Users\yeshw\AppData\Local\Temp\ipykernel_9404\1261022120.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



,id,name,city,county,country_code,activity_status,capabilities,latitude,longitude
0,2386,Al Jazri Lab,,Sharjah,AE,active,"[three_d_printing, cnc_milling, circuit_produc...",,
1,17,PiNG,Nantes,Pays de la Loire,FR,,[],47.218371,-1.553621
2,20,FabLab INSA Strasbourg,Strasbourg,Alsace,FR,,[],48.583148,7.747882
3,2852,FabLab Solidaire Orange Digital Center Côte d'...,Abidjan,Abidjan/Plateau/Indénié,CI,active,"[three_d_printing, cnc_milling, laser, vinyl_c...",,
4,2963,FabLab Caxias do Sul,Caxias do Sul,Rio Grande do Sul,BR,active,"[three_d_printing, cnc_milling, circuit_produc...",-29.139272,-51.171602


In [7]:
df.columns

Index(['id', 'name', 'city', 'county', 'country_code', 'activity_status',
       'capabilities', 'latitude', 'longitude'],
      dtype='object')

In [8]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Ensure your DataFrame 'df' exists and has the required columns:
# ['name', 'city', 'county', 'country_code', 'activity_status', 'capabilities']

# Convert list to string for display in table
df['capabilities_str'] = df['capabilities'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

table_columns = ['name', 'city', 'county', 'country_code', 'activity_status', 'capabilities_str']

# Initialize the dashboard
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    html.H2("FabLab Dashboard", style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#333'}),

    # Filters
    html.Div([
        html.Div([html.Label("Name", style={'fontWeight': 'bold'}),
                  dcc.Dropdown(id='filter-name', placeholder='Select name')], style={'marginBottom': '10px'}),

        html.Div([html.Label("Country", style={'fontWeight': 'bold'}),
                  dcc.Dropdown(id='filter-country', placeholder='Select country')], style={'marginBottom': '10px'}),

        html.Div([html.Label("Country Code", style={'fontWeight': 'bold'}),
                  dcc.Dropdown(id='filter-country-code', placeholder='Select country code')], style={'marginBottom': '10px'}),

        html.Div([html.Label("City", style={'fontWeight': 'bold'}),
                  dcc.Dropdown(id='filter-city', placeholder='Select city')], style={'marginBottom': '10px'}),

        html.Div([html.Label("Activity Status", style={'fontWeight': 'bold'}),
                  dcc.Dropdown(id='filter-status', placeholder='Select activity status')], style={'marginBottom': '10px'}),
    ], style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(auto-fit, minmax(250px, 1fr))',
        'gap': '15px',
        'padding': '15px',
        'backgroundColor': '#ffffff',
        'borderRadius': '8px',
        'boxShadow': '0 2px 5px rgba(0,0,0,0.1)',
        'marginBottom': '20px'
    }),

    # Data table
    dash_table.DataTable(
        id='table',
        columns=[{"name": i.replace('_str','').replace('_',' ').title(), "id": i} for i in table_columns],
        data=df[table_columns].to_dict('records'),
        page_size=10,
        sort_action="native",
        style_table={'overflowX': 'auto', 'border': 'none'},
        style_cell={'textAlign': 'left', 'padding': '8px', 'fontFamily': 'Arial, sans-serif',
                    'fontSize': '14px', 'backgroundColor': '#ffffff', 'color': '#333'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#f0f0f0', 'color': '#333'},
        style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#f7f7f7'}],
        fill_width=True
    ),

    # Pie chart
    dcc.Graph(id='capabilities-pie', style={'marginTop': '30px'})
], style={'backgroundColor': '#ffffff', 'padding': '20px', 'minHeight': '100vh'})


# Update dropdown options dynamically
@app.callback(
    Output('filter-name', 'options'),
    Output('filter-country', 'options'),
    Output('filter-country-code', 'options'),
    Output('filter-city', 'options'),
    Output('filter-status', 'options'),
    Input('filter-name', 'value'),
    Input('filter-country', 'value'),
    Input('filter-country-code', 'value'),
    Input('filter-city', 'value'),
    Input('filter-status', 'value')
)
def update_dropdowns(name, country, country_code, city, status):
    filtered = df.copy()
    if name:
        filtered = filtered[filtered['name'] == name]
    if country:
        filtered = filtered[filtered['county'] == country]
    if country_code:
        filtered = filtered[filtered['country_code'] == country_code]
    if city:
        filtered = filtered[filtered['city'] == city]
    if status:
        filtered = filtered[filtered['activity_status'] == status]

    return (
        [{'label': c, 'value': c} for c in sorted(filtered['name'].dropna().unique())],
        [{'label': c, 'value': c} for c in sorted(filtered['county'].dropna().unique())],
        [{'label': c, 'value': c} for c in sorted(filtered['country_code'].dropna().unique())],
        [{'label': c, 'value': c} for c in sorted(filtered['city'].dropna().unique())],
        [{'label': c, 'value': c} for c in sorted(filtered['activity_status'].dropna().unique())]
    )


# Update table and pie chart based on filters
@app.callback(
    Output('table', 'data'),
    Output('capabilities-pie', 'figure'),
    Input('filter-name', 'value'),
    Input('filter-country', 'value'),
    Input('filter-country-code', 'value'),
    Input('filter-city', 'value'),
    Input('filter-status', 'value')
)
def update_table_and_chart(name, country, country_code, city, status):
    filtered = df.copy()
    if name:
        filtered = filtered[filtered['name'] == name]
    if country:
        filtered = filtered[filtered['county'] == country]
    if country_code:
        filtered = filtered[filtered['country_code'] == country_code]
    if city:
        filtered = filtered[filtered['city'] == city]
    if status:
        filtered = filtered[filtered['activity_status'] == status]

    # Prepare pie chart
    all_caps = []
    for caps in filtered['capabilities']:
        if isinstance(caps, list):
            all_caps.extend(caps)

    if all_caps:
        cap_counts = pd.Series(all_caps).value_counts().sort_values(ascending=False)
        labels = [f"{name} → {count}" for name, count in zip(cap_counts.index, cap_counts.values)]

        fig = px.pie(
            values=cap_counts.values,
            names=labels,
            title="Capabilities Distribution",
            color=cap_counts.index,
            color_discrete_sequence=px.colors.qualitative.Vivid
        )

        # Professional styling + tooltip
        fig.update_traces(
            textinfo='label+percent',
            textposition='outside',
            textfont_size=14,
            marker=dict(line=dict(color='#ffffff', width=2)),
            hovertemplate="<b>%{label}</b><extra></extra>",
            hoverlabel=dict(
                bgcolor="white",
                font_size=14,
                font_family="Arial",
                font_color="black"
            )
        )

        fig.update_layout(
            title=dict(font_size=20, x=0.5),
            showlegend=False,
            margin=dict(t=50, b=20, l=20, r=150),
            paper_bgcolor="#ffffff",
            plot_bgcolor="#ffffff"
        )
    else:
        fig = px.pie(values=[1], names=["No Capabilities"], title="Capabilities Distribution")
        fig.update_traces(
            textinfo='label',
            hovertemplate="<b>No Capabilities</b><extra></extra>",
            hoverlabel=dict(
                bgcolor="white",
                font_size=14,
                font_family="Arial",
                font_color="black"
            )
        )
        fig.update_layout(
            title=dict(font_size=20, x=0.5),
            showlegend=False,
            paper_bgcolor="#ffffff",
            plot_bgcolor="#ffffff"
        )

    return filtered[table_columns].to_dict('records'), fig


# Run the dashboard in a new browser window
app.run(mode='external', debug=True, port=8050)


e:\hemanth\project\New_production_institute\IOT_Stack\Industrial dashboard\myenv\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

